In [96]:
import pandas as pd
import numpy as np
from function import analise_estoque, analise_estoque_grupo
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

        '005 - Calil',
        '007 - Rio Vermelho',
        '008 - Vargem',
        '009 - Canasvieiras',
        '010 - Upa',
        '011 - Trindade',

In [72]:
def vendas_grupo(grupo):
    df = pd.read_excel(f'planilhas/vendas/vendedores/vendas_{grupo}.xls',header=10, usecols=('B,D,F,T,Z,AF,AM,BC') )
    df = df.set_axis(['venda','filial','pagamento','data','hora','cupom','vendedor','valor_liquido'], axis=1)
    df['vendedor'] = df['vendedor'].shift(-1)
    df['valor_liquido'] = df['valor_liquido'].shift(-2)
    df.dropna(inplace=True)
    df['filial'] = df['filial'].astype(int)
    df['cupom'] = df['cupom'].astype(int)
    df['vendedor'] = df['vendedor'].astype(int)
    return df

In [77]:
df_grupo = vendas_grupo('perfumaria')
df_grupo

,venda,filial,pagamento,data,hora,cupom,vendedor,valor_liquido
1,145543,1,CARTAO / PIX,2024-08-01,1970-01-01 09:30:00,88979,10,6.99
4,145547,1,CARTAO / PIX,2024-08-01,1970-01-01 11:32:00,88986,10,19.00
7,145548,1,CARTAO / PIX,2024-08-01,1970-01-01 11:11:00,88983,87,11.99
10,145554,1,CARTAO / PIX,2024-08-01,1970-01-01 11:49:00,88989,10,2.99
13,145556,1,CARTAO / PIX,2024-08-01,1970-01-01 13:09:00,88992,87,7.99
...,...,...,...,...,...,...,...,...
18769,1304,11,CARTAO / PIX,2024-09-02,1970-01-01 20:44:00,1234,112,14.99
18772,1309,11,CARTAO / PIX,2024-09-02,1970-01-01 21:22:00,1238,112,182.99
18775,1312,11,CARTAO / PIX,2024-09-02,1970-01-01 22:08:00,1242,116,29.99
18778,1313,11,CARTAO / PIX,2024-09-02,1970-01-01 22:14:00,1243,112,7.99


In [95]:
vendedor = df_grupo.query('vendedor == 10')
df2 = vendedor.groupby(['data'])
dataframe_tkm = df2[['valor_liquido']].sum() / df2[['valor_liquido']].count()
dataframe_tkm['media_7d'] = dataframe_tkm['valor_liquido'].rolling(7).mean()
dataframe_tkm

,valor_liquido,media_7d
data,,
2024-08-01,12.245000,NaN
2024-08-02,22.255000,NaN
2024-08-03,37.857500,NaN
2024-08-04,22.035000,NaN
2024-08-06,64.940000,NaN
2024-08-07,19.078182,NaN
2024-08-08,13.785000,27.456526
2024-08-09,69.980000,35.704383
2024-08-10,5.990000,33.380812


In [97]:
def graf_plotly(data_frame, titulo):
    fig = go.Figure()
    fig.update_layout(
    title= f'{titulo}', 

    xaxis=dict(
        rangeslider= dict(visible=True),
        type='date',
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        
        showgrid=False,
        zeroline=True,
        showline=True,
        showticklabels=True,
    ),
    autosize=True,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white',
    legend= dict(
        font=dict(
            family='Arial',
            size=9)
    )
    )
    count = 0
    for i in data_frame.columns:
        if count < 2:
            count += 1
            lines = fig.add_trace(go.Scatter(x=data_frame.index, y=data_frame[f'{i}'], name= f"{i}", mode="markers+lines", visible=True))
        elif count == 2:
            count += 1
            lines = fig.add_trace(go.Scatter(x=data_frame.index, y=data_frame[f'{i}'], name= f"{i}", mode="lines", visible=True))
        elif count >2:
            count += 1
            lines = fig.add_trace(go.Scatter(x=data_frame.index, y=data_frame[f'{i}'], name= f"{i}", mode="lines", visible='legendonly'))
    return lines

In [98]:
graf_plotly(dataframe_tkm,'Evolução do Ticket Médio')

In [17]:
df_relacao_vendas = pd.read_excel('planilhas/vendas/vendedores/relacao_vendas.xls', header=10, usecols=('B,E,G,Q,U,AA,AB,AI,AL,AM,AP,AS'))
df_relacao_vendas = df_relacao_vendas.set_axis(['cod_venda','filial','forma_pagamento','data','hora','cupom','cliente','vendedor','valor_bruto','%desconto','valor_desconto','valor_liquido'], axis=1)[0:-3]
df_relacao_vendas['vendedor'] = df_relacao_vendas['vendedor'].astype(int)
df_relacao_vendas.head()

,cod_venda,filial,forma_pagamento,data,hora,cupom,cliente,vendedor,valor_bruto,%desconto,valor_desconto,valor_liquido
0,145540,1.0,CARTAO / PIX,2024-08-01,1970-01-01 07:31:00,88976.0,NaN,10,53.61,0.00,0.0,53.61
1,145541,1.0,DINHEIRO,2024-08-01,1970-01-01 07:32:00,88977.0,NaN,10,12.00,16.67,-2.0,10.00
2,145542,1.0,CARTAO / PIX,2024-08-01,1970-01-01 07:48:00,88978.0,NaN,10,44.37,0.00,0.0,44.37
3,145543,1.0,CARTAO / PIX,2024-08-01,1970-01-01 09:30:00,88979.0,NaN,10,27.68,11.20,-3.1,24.58
4,145544,1.0,CARTAO / PIX,2024-08-01,1970-01-01 10:40:00,88982.0,NaN,10,42.99,0.00,0.0,42.99


In [34]:
df_relacao_vendas['cod_venda'].count()

16745

In [35]:
a = df_relacao_vendas['cliente'].count()

In [36]:
b = df_relacao_vendas['cliente'].isna().sum()

In [37]:
a/(a+b)*100

10.606151089877576

In [38]:
df_colaboradores = pd.read_excel('planilhas/vendas/vendedores/lista_colaboradores.xls', header=8,usecols=('B,C,E'))[0:-5]
df_colaboradores = df_colaboradores.set_axis(['codigo','colaborador','admissao'], axis=1)
df_colaboradores['cod_nome_colab'] = df_colaboradores['codigo'].astype(str) + ' - '+ df_colaboradores['colaborador']
a = df_colaboradores['cod_nome_colab'].unique()

In [41]:
for i in a:
    print(i)

1 -  PEDRINHO BRITO
2 -  LUIZ ANDRE FERREIRA
6 -  EMERSON SOARES
9 -  SILVANIA MATOS
10 -  OTAVIO ALVES DE ALMEIDA
11 -  SHANDRICA SOARES RODRIGUES
33 -  SHIRLEY MARLENE SANTOS
38 -  CARMEN LUCIA SILVEIRA GONCALVES
39 -  PAULO MATOS
42 -  ERINEU FRANCISCO CORDEIRO
43 -  EDSON ROSARIO
45 -  MICHELLE ENCK
52 -  DIEGO NUNES
65 -  MIRELLY DE CASSIA PALACIOS FERREIRA
66 -  ELAINE COELHO
67 -  RICARDO RAMOS CASSOL
70 -  DIRCEMARA SOARES CARVALHO
72 -  GENECI MARIA DE CASTRO SILVEIRA
73 -  MARIELE DE SOUZA LEITE
74 -  LUCAS BORGES
75 -  FABBIO AUGUSTO BURGHERA DOS SANTOS
79 -  LUANA SARAIVA
83 -  CARLOS ALEXANDRE DA CRUZ GOES
84 -  TANIA  ANDREIA SCHULTZ BEDATTY
87 -  MICHAEL DOUGLAS GOMES SILVERIO
88 -  JAQUELINE DA SILVA SOUZA
89 -  PAULA ELIANA VILLEGAS DEL RIO
90 -  CRISZIELLY PRATES SCHROEDER
93 -  FLAVIA BATISTA VILACA
94 -  TAYANE ALVES FELIPE
95 -  DANIEL DA CRUZ SANTOS ROSA
97 -  GABRIELLY DA SILVA DE BARCELLOS BRANDAO 
98 -  ANA PAULA ZUCHI CORDEIRO
100 -  CRISTIANE CAVALHEIRO
101 -

In [21]:
#vendas genericos/similares
df_gensim = pd.read_excel('planilhas/vendas/vendedores/vendas_genericos_similares.xls', header=9)
#df_gensim = df_gensim.set_axis(['codigo','vendedor','valor_liquido'],axis=1)
df_gensim.head(30)

,Unnamed: 0,Unnamed: 1,Data Emisssão,Unnamed: 3,Unnamed: 4,Qtd. Vendas,Valor Custo,Valor Vendas,Margem Lucro
0,NaN,NaN,Vendedor:,2.0,LUIZ ANDRE FERREIRA,NaN,NaN,NaN,NaN
1,NaN,NaN,2024-08-04 00:00:00,NaN,NaN,1.0,14.62,35.50,142.82
2,NaN,NaN,2024-08-16 00:00:00,NaN,NaN,1.0,5.86,4.08,-30.38
3,NaN,NaN,2024-08-27 00:00:00,NaN,NaN,1.0,12.51,11.69,-6.55
4,NaN,NaN,2024-08-28 00:00:00,NaN,NaN,1.0,3.65,8.23,125.48
5,NaN,NaN,Total Vendedor:,NaN,NaN,4.0,36.64,59.50,62.39
6,NaN,NaN,Vendedor:,6.0,EMERSON SOARES,NaN,NaN,NaN,NaN
7,NaN,NaN,2024-08-09 00:00:00,NaN,NaN,1.0,7.59,16.99,123.85
8,NaN,NaN,2024-08-12 00:00:00,NaN,NaN,1.0,8.3,1.00,-87.95
9,NaN,NaN,2024-08-15 00:00:00,NaN,NaN,2.0,20.78,47.99,130.94


In [44]:
df = df_gensim

In [45]:
# Passo 1: Identificar as linhas com informações de vendedores e limpar colunas desnecessárias
vendedor = None
vendedor_col = []

for i in range(len(df)):
    # Verifica se a linha tem o texto 'Vendedor:'
    if df.iloc[i].str.contains('Vendedor:', na=False).any():
        # Extrai o nome e código do vendedor
        qtd_vendas = df.iloc[i][df.iloc[i].str.contains(r'^\d+', na=False, regex=True)].values
        nome_vendedor = df.iloc[i][~df.iloc[i].isna()].values[-1]
        vendedor = f"{qtd_vendas[0] if qtd_vendas.size > 0 else ''} - {nome_vendedor}"
    vendedor_col.append(vendedor)

# Adiciona a coluna 'Vendedor' ao DataFrame
df['Vendedor'] = vendedor_col

# Passo 2: Remover as linhas de totais e vazias, e limpar a coluna de data
df = df[~df['Data Emisssão'].str.contains('Total Vendedor:', na=False)]
df = df.dropna(subset=['Data Emisssão', 'Qtd. Vendas', 'Valor Custo', 'Valor Vendas', 'Margem Lucro'])

# Passo 3: Converter a coluna 'Data Emisssão' para formato de data
df['Data Emisssão'] = pd.to_datetime(df['Data Emisssão'], errors='coerce')
df = df.dropna(subset=['Data Emisssão'])

# Passo 4: Pivotar o dataframe para que cada vendedor tenha sua coluna com valores de vendas
df_pivot = df.pivot_table(index='Data Emisssão', columns='Vendedor', values='Valor Vendas', aggfunc='sum')

# Passo 5: Resetar o índice para ter 'Data Emisssão' como uma coluna normal
df_pivot = df_pivot.reset_index()

# Exibir o dataframe resultante
df_pivot

Vendedor,Data Emisssão,- - ANA PAULA ZUCHI CORDEIRO,- - BARBARA SANDRE,- - CAIO FELIPE RIBEIRO DE BRITO,- - CARLOS ALEXANDRE DA CRUZ GOES,- - CARLOS EDUARDO BASTOS DO NASCIMENTO,- - CARMEN LUCIA SILVEIRA GONCALVES,- - CLEVERSON LUIS LORENSSETTI,- - CRISTIANE CAVALHEIRO,- - CRISZIELLY PRATES SCHROEDER,...,- - MICHAEL DOUGLAS GOMES SILVERIO,- - MICHELLE ENCK,- - OTAVIO ALVES DE ALMEIDA,- - PAULA ELIANA VILLEGAS DEL RIO,- - PAULO MATOS,- - RICARDO RAMOS CASSOL,- - SHANDRICA SOARES RODRIGUES,- - SHIRLEY MARLENE SANTOS,- - SILVANIA MATOS,- - TAYANE ALVES FELIPE
0,2024-08-01,144.98,NaN,NaN,816.20,NaN,NaN,234.89,150.96,NaN,...,357.10,315.24,182.11,146.96,NaN,451.20,103.81,545.23,1189.46,483.52
1,2024-08-02,NaN,190.07,NaN,353.81,NaN,1467.72,346.64,69.59,NaN,...,232.60,527.42,214.39,262.95,NaN,NaN,152.69,20.98,477.74,274.12
2,2024-08-03,NaN,63.80,NaN,987.80,NaN,676.75,218.52,156.96,NaN,...,NaN,807.00,365.24,87.89,NaN,425.22,NaN,24.99,822.87,282.21
3,2024-08-04,NaN,NaN,NaN,436.26,NaN,650.14,NaN,NaN,NaN,...,NaN,449.83,153.52,234.13,NaN,NaN,NaN,NaN,17.00,865.05
4,2024-08-05,NaN,89.96,NaN,597.49,NaN,572.59,451.50,249.93,NaN,...,113.29,NaN,39.90,82.27,NaN,149.89,654.23,347.69,190.27,NaN
5,2024-08-06,NaN,255.46,NaN,170.65,NaN,834.97,366.35,NaN,NaN,...,130.99,501.83,612.69,327.93,NaN,306.36,75.45,64.49,254.97,300.33
6,2024-08-07,250.75,578.70,NaN,690.49,NaN,NaN,NaN,244.93,NaN,...,229.22,427.19,516.69,NaN,NaN,604.80,156.46,554.66,1365.11,538.73
7,2024-08-08,350.58,NaN,NaN,292.29,NaN,820.20,399.71,284.03,NaN,...,NaN,468.01,299.62,135.91,NaN,418.74,9.90,116.45,119.98,979.68
8,2024-08-09,94.98,368.23,NaN,515.46,NaN,1036.72,579.28,164.96,NaN,...,NaN,426.06,538.12,61.98,NaN,NaN,234.85,620.12,960.01,812.77
9,2024-08-10,204.55,230.68,NaN,1089.10,NaN,578.26,334.60,171.16,NaN,...,25.98,433.63,358.81,62.57,NaN,NaN,27.99,846.89,807.85,NaN
